Here a vector store solution will be integrated into a RAG pipeline and it will be evaluated using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics.
 
- RAG pipeline will be numerically evaluated.

# Questions:
- Should the pipeline pass these objectives? Is the judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for the use case?
- If the vectorstore-as-a-memory component is left in our chain, will it still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance?

### **Environment Setup:**

In [2]:
# %pip install -q langchain requests sentence-transformers gradio rich
# %pip install -q faiss ragas

## If a typing-extensions issue is encountered, restart your runtime and try again

from functools import partial
from huggingface_hub import list_models
import json
from langchain_core.embeddings.embeddings import Embeddings
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables.config import RunnableConfig
import os
from pydantic import BaseModel, ConfigDict, PrivateAttr
from rich.console import Console
from rich.style import Style
import requests
from sentence_transformers import SentenceTransformer
from typing import List, Optional

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

# Gemini API Configuration
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent"
# Secret GEMINI_API_KEY environment variable to be set beforehand
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY environment variable not set.")
    exit()

class SentenceTransformersEmbeddings(BaseModel, Embeddings):
    """Wrapper for Gemini Embeddings."""

    model_config = ConfigDict(arbitrary_types_allowed=True)
    model: str = "Alibaba-NLP/gte-base-en-v1.5"
    embedder: SentenceTransformer = None
    
    def __init__(self, model: str = "Alibaba-NLP/gte-base-en-v1.5", trust_remote_code: Optional[bool] = False):
        """
        Initialize the SentenceTransformersEmbeddings class using a SentenceTransformer model.
        :param model: Name of the pre-trained model to load from SentenceTransformers.
        """
        super().__init__()
        self.embedder = SentenceTransformer(model, trust_remote_code=trust_remote_code)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        Generate embeddings for a list of texts.
        :param texts: List of input texts.
        :return: List of embeddings (one for each text).
        """
        if not texts:
            pprint2("")
            raise ValueError("Input texts list cannot be empty.")
        return self.embedder.encode(texts, convert_to_tensor=False).tolist()

    def embed_query(self, text: str) -> List[float]:
        """
        Generate embedding for a single query.
        :param text: Input query text.
        :return: Embedding for the query as a list of floats.
        """
        if not text:
            raise ValueError("Input text cannot be empty.")
        return self.embedder.encode(text, convert_to_tensor=False).tolist()

    @classmethod
    def get_available_models(cls, search_term: str = "sentence-transformers"):
        """
        Dynamically fetch available SentenceTransformer models from Hugging Face Model Hub.
        :param search_term: Search term to filter relevant models (default: 'sentence-transformers').
        :return: List of model names.
        """
        models = list_models(author=search_term)
        return [model.modelId for model in models]

class GeminiChat(BaseChatModel):
    """Wrapper for Gemini Chat."""

    model: str = "gemini-1.5-flash"
    temperature: float = 1.0
    max_tokens: Optional[int] = None

    # Declare private attributes
    _api_url: str = PrivateAttr()
    _headers: dict = PrivateAttr()
    
    def __init__(self, model: str = "gemini-1.5-flash", temperature: float = 1.0, max_tokens: Optional[int] = None):
        super().__init__(model=model, temperature=temperature, max_tokens=max_tokens)
        
        # Initialize private attributes
        self._api_url = f"{GEMINI_API_URL}"
        self._headers = {
            "Content-Type": "application/json"
        }

    def _llm_type(self):
        return "gemini-1.5-flash"
    
    def _generate(self, prompt: ChatPromptValue, stop: Optional[List[str]] = None) -> str:
        payload = {
            "contents": [
                {
                    "parts": [{"text": prompt.messages[0].content}]
                }
            ]
        }
        if self.max_tokens is not None:  # Include max_tokens only if it's valid
            payload["max_tokens"] = self.max_tokens
        response = requests.post(
            f"{self._api_url}?key={GEMINI_API_KEY}",
            headers=self._headers,
            json=payload
        )
        if response.status_code == 200:
            try:
                return response.json()["candidates"][0]["content"]["parts"][0]["text"]
            except Exception as e:
                pprint2(f"Error parsing response JSON: {repr(e)}")
                return ""
        else:
            try:
                error_data = response.json()
                pprint2(f"Error {response.status_code}: {error_data}")
            except Exception as e:
                pprint2(f"Error {response.status_code}: {repr(e)}")
            return ""
            
    def invoke(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self._generate(prompt, stop)

    def stream(self, input_text: ChatPromptValue, config: Optional[RunnableConfig]):
        # Simulate streaming by yielding tokens from the response
        response = self._generate(input_text)
        for token in response:
            yield token
    
    @property
    def _identifying_params(self):
        return {
            "model": self.model,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }

    @classmethod
    def get_available_models(cls):
        pprint2("GeminiChat does not support listing available models via API.")
        return []

# Instantiate SentenceTransformersEmbeddings and GeminiChat
embedder = SentenceTransformersEmbeddings(model="Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)
instruct_llm = GeminiChat(model="gemini-1.5-flash", temperature=1.0)

## 1. Pre-Release Evaluation

- **Typical Use Inspection:**

- **Edge Case Inspection:**

- **Progressive Rollout:**

## 2. LLM-as-a-Judge Formulation

- LLM simulates a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**Popular frameworks for off-the-shelf judge formulations:**
- [**RAGAs (RAG Assessment)**](https://docs.ragas.io/en/stable/)
- [**LangChain Evaluators**](https://python.langchain.com/docs/guides/evaluation/)

### 3. RAG For Conversation History

### 3.1 Constructing Our Vector Store Retriever
**Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.

Now we start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine.

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain.vectorstores import FAISS

conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

# Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 136 ms, sys: 123 ms, total: 259 ms
Wall time: 387 ms


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [6]:
from langchain_core.runnables import RunnableLambda
from langchain.document_transformers import LongContextReorder

## Utility Runnable/Method
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [8]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Beras lives in the arctic.

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

According to the conversation, the Rocky Mountains are a range of mountains that stretch across North America.

In [12]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Based on the conversation, the Rocky Mountains are in North America.  The provided text doesn't say whether they 
are close to California or not.

In [13]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Based on the conversation, Beras lives in the Arctic.  The Rocky Mountains are in North America.  The exact 
distance isn't specified, but it's a considerable distance given their differing locations.

### 3.2 Automatic Conversation Storage

Now we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.

In [16]:
from langchain_core.runnables import RunnableAssign
from operator import itemgetter

## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)

conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)
pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

That's great you're excited!  While we're talking about the Rocky Mountains,  I don't think ice cream was mentioned
in our previous conversation. Are you thinking of a different place?

Based on our conversation, it sounds like ice cream is your favorite food!

Oh, whoops!  My apologies! I must have gotten that wrong.  I guess I jumped to conclusions based on your excitement
about ice cream.  Honey sounds delicious too!

Well, you seem pretty excited about ice cream, so I'm going to guess that's still your favorite!  But I learned my 
lesson about jumping to conclusions!

## 4. RAG For Document Chunk Retrieval

The idea that data chunks can be embedded and searched through is flabbergasting. Applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance.


### 4.1 Loading And Chunking Documents

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - ReAct: Synergizing Reasoning and Acting in Language Models
 - High-Resolution Image Synthesis with Latent Diffusion Models
 - Learning Transferable Visual Models From Natural Language Supervision 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2023-03-10',
    'Title': 'ReAct: Synergizing Reasoning and Acting in Language Models',
    'Authors': 'Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Karthik Narasimhan, Yuan Cao',
    'Summary': 'While large language models (LLMs) have demonstrated impressive capabilities\nacross tasks in 
language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought 
prompting) and acting (e.g.\naction plan generation) have primarily been studied as separate topics. In 
this\npaper, we explore the use of LLMs to generate both reasoning traces and\ntask-specific actions in an 
interleaved manner, allowing for greater synergy\nbetween the two: reasoning traces help the model induce, track, 
and update\naction plans as well as handle exceptions, while actions allow it to interface\nwith external sources, 
such as knowledge bases or environments, to gather\nadditional information. We apply our approach, named ReAct, to 
a diverse set of\nlanguage and decision making tasks and demonstrate its effectiveness over\nstate-of-the-art 
baselines, as well as improved human interpretability and\ntrustworthiness over methods without reasoning or acting
components.\nConcretely, on question answering (HotpotQA) and fact verification (Fever),\nReAct overcomes issues of
hallucination and error propagation prevalent in\nchain-of-thought reasoning by interacting with a simple Wikipedia
API, and\ngenerates human-like task-solving trajectories that are more interpretable than\nbaselines without 
reasoning traces. On two interactive decision making\nbenchmarks (ALFWorld and WebShop), ReAct outperforms 
imitation and\nreinforcement learning methods by an absolute success rate of 34% and 10%\nrespectively, while being
prompted with only one or two in-context examples.\nProject site with code: https://react-lm.github.io'
}


Document 7
 - # Chunks: 52
 - Metadata: 


{
    'Published': '2022-04-13',
    'Title': 'High-Resolution Image Synthesis with Latent Diffusion Models',
    'Authors': 'Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer',
    'Summary': 'By decomposing the image formation process into a sequential application of\ndenoising 
autoencoders, diffusion models (DMs) achieve state-of-the-art\nsynthesis results on image data and beyond. 
Additionally, their formulation\nallows for a guiding mechanism to control the image generation process 
without\nretraining. However, since these models typically operate directly in pixel\nspace, optimization of 
powerful DMs often consumes hundreds of GPU days and\ninference is expensive due to sequential evaluations. To 
enable DM training on\nlimited computational resources while retaining their quality and flexibility,\nwe apply 
them in the latent space of powerful pretrained autoencoders. In\ncontrast to previous work, training diffusion 
models on such a representation\nallows for the first time to reach a near-optimal point between 
complexity\nreduction and detail preservation, greatly boosting visual fidelity. By\nintroducing cross-attention 
layers into the model architecture, we turn\ndiffusion models into powerful and flexible generators for general 
conditioning\ninputs such as text or bounding boxes and high-resolution synthesis becomes\npossible in a 
convolutional manner. Our latent diffusion models (LDMs) achieve\na new state of the art for image inpainting and 
highly competitive performance\non various tasks, including unconditional image generation, semantic 
scene\nsynthesis, and super-resolution, while significantly reducing computational\nrequirements compared to 
pixel-based DMs. Code is available at\nhttps://github.com/CompVis/latent-diffusion .'
}


Document 8
 - # Chunks: 155
 - Metadata: 


{
    'Published': '2021-02-26',
    'Title': 'Learning Transferable Visual Models From Natural Language Supervision',
    'Authors': 'Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal, Girish 
Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever',
    'Summary': 'State-of-the-art computer vision systems are trained to predict a fixed set\nof predetermined 
object categories. This restricted form of supervision limits\ntheir generality and usability since additional 
labeled data is needed to\nspecify any other visual concept. Learning directly from raw text about images\nis a 
promising alternative which leverages a much broader source of\nsupervision. We demonstrate that the simple 
pre-training task of predicting\nwhich caption goes with which image is an efficient and scalable way to 
learn\nSOTA image representations from scratch on a dataset of 400 million (image,\ntext) pairs collected from the 
internet. After pre-training, natural language\nis used to reference learned visual concepts (or describe new ones)
enabling\nzero-shot transfer of the model to downstream tasks. We study the performance\nof this approach by 
benchmarking on over 30 different existing computer vision\ndatasets, spanning tasks such as OCR, action 
recognition in videos,\ngeo-localization, and many types of fine-grained object classification. The\nmodel 
transfers non-trivially to most tasks and is often competitive with a\nfully supervised baseline without the need 
for any dataset specific training.\nFor instance, we match the accuracy of the original ResNet-50 on 
ImageNet\nzero-shot without needing to use any of the 1.28 million training examples it\nwas trained on. We release
our code and pre-trained model weights at\nhttps://github.com/OpenAI/CLIP.'
}

### 4.2 Constructing Document Vector Stores

In [34]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunk, embedder) for doc_chunk in docs_chunks]

Constructing Vector Stores
CPU times: user 3.74 s, sys: 7min 18s, total: 7min 21s
Wall time: 35min 46s


Constructing Vector Stores
CPU times: user 3.74 s, sys: 7min 18s, total: 7min 21s
Wall time: 35min 46s

In [ ]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to our embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

If you want to load pre-constructed docstore instead:

In [34]:
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
print(f"Loaded aggregate docstore with {len(docstore.docstore._dict)} chunks")

Loaded aggregate docstore with 571 chunks


### 4.3 Implementing RAG Chain

In [36]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
import gradio as gr
from operator import itemgetter

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_template("You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
)

stream_chain = chat_prompt | instruct_llm | StrOutputParser()

## Implementing the retrieval chain to make the system work

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## Retrieve history & context from convstore & docstore, respectively.
    ## Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history': itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context': itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
)

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

RAG, or Retrieval-Augmented Generation, is a method for answering questions using both a retriever and a generator.  Unlike some other approaches, it achieves state-of-the-art performance without needing expensive pre-training or extra components like a re-ranker or extractive reader [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks].  It's advantageous because it can generate answers even if the exact answer isn't verbatim in the retrieved documents; it can synthesize information from multiple documents to create a correct response, something extractive methods can't do [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks]. In fact, RAG can even generate correct answers even when the answer isn't in *any* of the retrieved documents [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks].  There are two main variations: RAG-Token and RAG-Sequence, with the latter requiring a different decoding approach due to its likelihood calculation [Retrieval

### 4.4 Gradio Chatbot

In [42]:
mssg = {"role": "user", "content": initial_msg}
chatbot = gr.Chatbot(value=[mssg], type='messages')
demo = gr.ChatInterface(chat_gen, chatbot=chatbot, type='messages').queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://b8adb8c80dc10192d6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b8adb8c80dc10192d6.gradio.live
Closing server running on port: 7860


### 4.5 Saving The Index

In [40]:
## Save and compress the index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

# !rm -rf docstore_index

a docstore_index
a docstore_index/index.faiss
a docstore_index/index.pkl


### 4.6 Testing The Index

In [133]:
os.environ["TOKENIZERS_PARALLELISM"] = "False"
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
testdocs = new_db.similarity_search("Testing")
print(testdocs[0].page_content[:1000])

x docstore_index/
x docstore_index/index.faiss
x docstore_index/index.pkl
.\n\u2022 Better characterizing biases in models, alerting other\nresearchers to areas of concern and areas for interven-\ntions.\n\u2022 Creating suites of tests to evaluate systems like CLIP\non, so we can better characterize model capabilities\nearlier in the development cycle.\n\u2022 Identifying potential failure modes and areas for further\nwork.\nWe plan to contribute to this work, and hope this analysis\nprovides some motivating examples for subsequent research.\n8. Related Work\nAny model that leverages written, spoken, signed or any\nother form of human language as part of its training signal\nis arguably using natural language as a source of supervi-\nsion. This is an admittedly extremely broad area and covers\nmost work in the \ufb01eld of distributional semantics including\ntopic models (Blei et al., 2003), word, sentence, and para-\ngraph vectors (Mikolov et al., 2013; Kiros et al., 2015; Le &\nMiko

## 5. Pairwise Evaluator

The following is a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/docs/guides/evaluation/examples/comparisons). 

**Preparing for RAG chain evaluation, we need to:**

- Pull in the document index.
- Recreate the RAG pipeline of choice.

**Implementing a judge formulation as follows:**

- Sample RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that that tests if the RAG chain outperforms a narrow chatbot with limited document access.

### 5.1. Pulling In Document Retrieval Index

In [139]:
## Make sure docstore_index.tgz is in working directory
from langchain_community.vectorstores import FAISS

# embedder = SentenceTransformersEmbeddings(model="Alibaba-NLP/gte-base-en-v1.5")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content.replace("\n", " ")}"
    )

## This printout just confirms that the store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

x docstore_index/
x docstore_index/index.faiss
x docstore_index/index.pkl


Constructed aggregate docstore with 571 chunks

Sample Chunk:

Paper: ReAct: Synergizing Reasoning and Acting in Language Models

Summary: While large language models (LLMs) have demonstrated impressive capabilities
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.
action plan generation) have primarily been studied as separate topics. In this
paper, we explore the use of LLMs to generate both reasoning traces and
task-specific actions in an interleaved manner, allowing for greater synergy
between the two: reasoning traces help the model induce, track, and update
action plans as well as handle exceptions, while actions allow it to interface
with external sources, such as knowledge bases or environments, to gather
additional information. We apply our approach, named ReAct, to a diverse set of
language and decision making tasks and demonstrate its effectiveness over
state-of-the-art baselines, as well as improved human interpretability and
trustwor

### 5.2. Pulling In RAG Chain 

In [217]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch
from operator import itemgetter

embedder = SentenceTransformersEmbeddings(model="Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

instruct_llm = GeminiChat(model="gemini-1.5-flash")

llm = instruct_llm | StrOutputParser()

chat_prompt = ChatPromptTemplate.from_template(
    "{input}"
    " Use the format:\n\nAnswer: (answer)\n\n"
    "Document Retrieval:\n{context}\n\n"
    "Question Again: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

## Chain 1 Specs: "Hello World" -> retrieval_chain 
long_reorder = RunnableLambda(LongContextReorder().transform_documents)
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
retrieval_chain = {'input': (lambda x: x)} | RunnableAssign({'context': context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
generator_chain = RunnableLambda(lambda x: RunnableAssign({'formatted_input': chat_prompt.invoke})) | RunnableLambda(lambda x: RunnableAssign({'output': RunnableLambda(lambda y: llm.invoke(y['formatted_input']))}))

rag_chain = retrieval_chain | generator_chain

## pprint(rag_chain.invoke("Tell me something interesting!"))
for token in [rag_chain.stream("Tell me something interesting!")]:
    for t in RunnableLambda(output_puller).invoke(token):
        print(t, end="")

Answer:  Researchers are developing a new type of question-answering task using Jeopardy! clues as a benchmark.  Because Jeopardy! questions require concise, factual statements that cleverly conceal an answer, generating them is a challenging test of a large language model's knowledge and generation capabilities.


### 5.3. Generating Synthetic Question-Answer Pairs

Now we implement the first few part of our evaluation routine:

- **Sample RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests if the RAG chain outperforms a narrow chatbot with limited document access.

In [231]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt1 = ChatPromptTemplate.from_template('{system}\n\n{input}')

for i in range(num_questions):
    doc1 = random.sample(docs, 1)[0]
    sys_msg = (
        "Use the documents provided to generate an interesting question."
        " Rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1 sentence, detailed)"
    )
    usr_msg = (
        f"Document: {format_chunk(doc1)}"
    )

    q = (simple_prompt1 | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [q]
    pprint2(f"Q {i+1}")
    pprint2(synth_questions[-1])
    print()

Q 1

Question:  Considering the ReAct model's success in integrating reasoning and action, as exemplified by its ability
to plan and execute actions like cleaning and placing a lettuce in the provided example, how might its interactive 
decision-making capabilities be further enhanced by incorporating more sophisticated environmental modeling and 
proactive planning beyond simple task completion?

Q 2

Question: Given the limitations of Large Language Models highlighted in the provided text, and the presented data 
on the performance of different mathematical formula structures within the MRKL system, how might the specific 
strengths and weaknesses of neuro-symbolic architectures like MRKL be leveraged to improve the accuracy and 
reliability of complex mathematical reasoning tasks in AI systems?

Q 3

Question:  Given the ReAct model's demonstrated success in mitigating hallucination and error propagation in 
question answering and fact verification by interacting with a Wikipedia API, what novel applications could be 
explored by integrating ReAct with other external knowledge bases or APIs, and how might this impact the model's 
reasoning and action capabilities across diverse tasks?

In [233]:
synth_answers = []

simple_prompt2 = ChatPromptTemplate.from_template('{input} {system}')

for i in range(num_questions):
    sys_msg = (
        " Use the format: \"Answer: (answer)\""
    )
    a = (simple_prompt2 | llm).invoke({'system': sys_msg, 'input': synth_questions[i]})
    synth_answers += [a]
    pprint2(f"A {i+1}")
    pprint(synth_answers[-1])
    print()

A 1

Answer: Enhancing ReAct's interactive decision-making capabilities beyond simple task completion requires 
incorporating more sophisticated environmental modeling and proactive planning. This could involve:  1) 
**Developing richer world models:**  Instead of relying on limited, immediate sensory input, integrating external 
knowledge bases, maps, or object-relationship graphs would allow ReAct to reason about the environment more 
comprehensively. This would enable proactive actions like anticipating obstacles before they are encountered (e.g.,
planning a route around a cluttered area before starting to clean). 2) **Hierarchical planning:**  Moving beyond 
flat sequences of actions, hierarchical planning would allow ReAct to break down complex tasks into sub-tasks, 
enabling more robust and flexible planning.  This would allow for contingency planning –  if an unexpected event 
occurs (e.g., the lettuce is unexpectedly slippery), the system can adapt its plan without restarting. 3) 
**Predictive modeling:**  Incorporating predictive models of the environment and the effects of actions would allow
ReAct to simulate potential outcomes and choose actions that maximize the likelihood of success. For example, 
predicting the trajectory of a moving object to avoid collisions while navigating. 4) **Reinforcement learning with
long-term rewards:**  Training the model with rewards that incentivize efficient and proactive behavior, extending 
beyond immediate task completion, would encourage the system to develop more sophisticated planning skills.  For 
example, rewarding efficient route planning or minimizing energy consumption. 5) **Uncertainty handling:**  
Incorporating probabilistic reasoning and uncertainty quantification would make the system more robust to noisy or 
incomplete information about the environment. This would allow ReAct to handle situations where perfect information
is unavailable and plan accordingly.  By incorporating these advancements, ReAct could evolve from a simple task 
executor to a proactive and adaptable agent capable of complex, long-term planning and decision-making in dynamic 
environments.

A 2

Answer:  Neuro-symbolic architectures like MRKL, by combining the strengths of neural networks (pattern 
recognition, generalization) and symbolic AI (explicit knowledge representation, logical reasoning), can mitigate 
the limitations of LLMs in complex mathematical reasoning.  The data on MRKL's performance with different formula 
structures highlights the importance of carefully designing the symbolic component to represent mathematical 
knowledge effectively.  To improve accuracy and reliability, we can leverage MRKL's strengths as follows:

1. **Improved Knowledge Representation:**  Use MRKL's symbolic reasoning capabilities to explicitly encode 
mathematical axioms, theorems, and rules of inference.  This addresses LLMs' weakness in lacking explicit 
understanding of mathematical concepts and relying on statistical correlations.  Analyzing the MRKL data to 
identify optimal formula structures for encoding different mathematical domains would be crucial.

2. **Hybrid Approach:**  Combine the neural network's capacity for learning complex patterns with the symbolic 
system's ability to perform deductive reasoning. The neural network can be trained on large datasets to learn 
approximations and heuristic strategies, while the symbolic component ensures correctness and consistency. This 
allows leveraging the strengths of both while minimizing their individual weaknesses.

3. **Explainability and Interpretability:**  The symbolic representation in MRKL allows for tracing the reasoning 
steps, providing explainability absent in purely neural network approaches.  This interpretability is crucial for 
debugging errors and building trust in the AI's mathematical conclusions.  Analyzing where MRKL falters (as 
revealed by the performance data) can guide the improvement of both symbolic representations and the integration 
with the neural component.

4. **Targeted Training:** Instead of relying on large-scale general training, focus on targeted training of the 
neural components of MRKL on specific sub-problems or mathematical domains, informed by the performance data on 
formula structures. This refined training can improve accuracy and address the limitations identified in the 
specific mathematical formula types.

5. **Iterative Refinement:** Use the performance data on MRKL as feedback to iteratively refine both the symbolic 
knowledge base and the neural network's architecture and training methodology. This continuous improvement cycle is
key to addressing the limitations of both approaches and maximizing the combined strengths.


In essence, a well-designed neuro-symbolic system like MRKL can create a robust and reliable AI system for complex 
mathematical reasoning by leveraging its capacity for both learning and logical deduction, provided that the 
symbolic component is carefully designed and the interaction between symbolic and neural components is optimized, 
informed by data-driven insights.

A 3

Answer: Integrating ReAct with other external knowledge bases and APIs beyond Wikipedia could unlock numerous novel
applications and significantly enhance its reasoning and action capabilities.  Here are some examples:

**Novel Applications:**

* **Scientific Discovery and Hypothesis Generation:**  ReAct could be integrated with scientific databases like 
PubMed, Reaxys, or arXiv.  This would allow it to formulate hypotheses, design experiments, and interpret results 
by referencing relevant literature and data, potentially accelerating scientific breakthroughs.  The model could 
reason about complex biological pathways, chemical reactions, or astronomical phenomena by accessing and processing
information from these specialized sources.

* **Financial Modeling and Risk Assessment:** Combining ReAct with financial APIs (e.g., accessing real-time market
data, company financials, economic indicators) could enable sophisticated financial modeling, risk assessment, and 
investment strategy generation. The model could analyze market trends, predict future performance, and make 
informed investment decisions based on the data it retrieves and processes.

* **Personalized Healthcare and Medical Diagnosis Support:**  Integrating ReAct with Electronic Health Records 
(EHR) systems and medical knowledge bases could assist doctors in diagnosis and treatment planning.  The model 
could analyze patient data, identify potential diagnoses, suggest relevant treatments, and even anticipate 
potential complications based on its access to medical literature and patient histories. However, ethical 
considerations regarding patient privacy and data security would need careful attention.

* **Legal Research and Due Diligence:**  Access to legal databases (e.g., Westlaw, LexisNexis) would allow ReAct to
perform legal research, identify relevant case law, and assist in due diligence processes. This could significantly
streamline legal workflows and improve the accuracy of legal advice.

* **Supply Chain Optimization and Logistics:**  Integration with real-time logistics and supply chain data APIs 
could allow ReAct to optimize inventory management, predict disruptions, and suggest efficient routing strategies. 
This could lead to cost savings and increased efficiency in supply chains.

**Impact on Reasoning and Action Capabilities:**

Integrating ReAct with diverse external knowledge bases would:

* **Expand its knowledge domain:**  ReAct's capabilities would no longer be limited to the information available 
within Wikipedia.  It could access and process information from specialized domains, significantly broadening its 
scope.
* **Improve reasoning accuracy:**  Access to more comprehensive and up-to-date information would reduce 
hallucination and improve the accuracy of its reasoning and conclusions.
* **Enable more complex reasoning tasks:**  The ability to interact with multiple data sources would allow ReAct to
perform more complex reasoning tasks that require synthesizing information from diverse sources.
* **Enhance action capabilities:** The model's ability to directly interact with and manipulate external systems 
(e.g., making API calls to update databases, trigger actions in a robotic system) would enable more sophisticated 
actions.


However, challenges remain: ensuring data quality, managing API rate limits, dealing with conflicting information 
from different sources, and addressing potential biases in the data accessed are crucial considerations for 
successful integration.

### 5.4. Answer The Synthetic Questions

Now we implement the third part of our evaluation routine:

- Sample RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests if the RAG chain outperforms a narrow chatbot with limited document access.

In [235]:
rag_answers = []
contexts = []
for i, q in enumerate(synth_questions):
    outp = rag_chain.invoke(q)
    rag_answer = outp.get('output', '')
    rag_answers += [rag_answer]
    context = outp.get('context', '')
    contexts += [context] 
    pprint2(f"Q {i+1}\n", q, "", sep="\n")
    pprint(f"RAG Answer: \n\n{rag_answer}", "", sep='\n')

Q 1

Question:  Considering the ReAct model's success in integrating reasoning and action, as exemplified by its ability
to plan and execute actions like cleaning and placing a lettuce in the provided example, how might its interactive 
decision-making capabilities be further enhanced by incorporating more sophisticated environmental modeling and 
proactive planning beyond simple task completion?

RAG Answer: 

Answer: Enhancing ReAct's interactive decision-making capabilities beyond simple task completion requires 
incorporating more sophisticated environmental modeling and proactive planning.  This could involve:

1. **Developing richer internal representations of the environment:**  Instead of relying on simple descriptions, 
ReAct could benefit from integrating more detailed, structured environmental models. This might involve using 
graph-based representations to capture object relationships, spatial layouts, and potential interactions.  This 
would allow for more complex reasoning about the consequences of actions and more robust planning.

2. **Implementing proactive planning mechanisms:** Currently, ReAct's planning seems largely reactive.  More 
sophisticated planning algorithms, such as Monte Carlo Tree Search (MCTS) or hierarchical planning, could be 
incorporated to explore potential action sequences and evaluate their expected outcomes before execution.  This 
proactive approach would enable anticipation of future needs and potential obstacles, leading to more efficient and
robust task completion.

3. **Integrating temporal reasoning:**  Adding temporal reasoning capabilities would allow ReAct to consider the 
timing of actions and their dependencies. For instance, understanding that cleaning the lettuce must precede 
placing it on the table is a simple example; more complex scenarios may require sophisticated scheduling and 
resource management.

4. **Incorporating uncertainty and risk assessment:**  Real-world environments are inherently uncertain.  
Augmenting ReAct with probabilistic models and risk assessment mechanisms would allow it to handle unexpected 
events and make more robust decisions under uncertainty. For example, it could plan for alternative actions if a 
required object is unavailable.

5. **Leveraging external knowledge bases:**  Integrating access to broader knowledge bases beyond simple APIs could
provide ReAct with more context and information for planning.  This could include knowledge about object 
properties, typical action sequences, or potential hazards.


By integrating these enhancements, ReAct could move beyond simple task completion towards more complex, adaptive, 
and proactive interaction with the environment.  It would then be able to handle unforeseen circumstances, plan for
contingencies, and optimize its actions for efficiency and robustness.

Q 2

Question: Given the limitations of Large Language Models highlighted in the provided text, and the presented data 
on the performance of different mathematical formula structures within the MRKL system, how might the specific 
strengths and weaknesses of neuro-symbolic architectures like MRKL be leveraged to improve the accuracy and 
reliability of complex mathematical reasoning tasks in AI systems?

RAG Answer: 

Answer: The provided text highlights limitations of Large Language Models (LLMs), such as a lack of access to 
current/proprietary information and an inability to reason symbolically.  MRKL, a neuro-symbolic architecture, 
addresses these weaknesses by combining LLMs with external knowledge sources and discrete reasoning modules.  To 
improve accuracy and reliability in complex mathematical reasoning, MRKL's strengths can be leveraged as follows:

1. **Symbolic Reasoning for Precise Calculations:**  LLMs excel at natural language processing but struggle with 
precise, symbolic manipulation needed for complex mathematics.  MRKL's discrete reasoning modules can handle this, 
ensuring accurate execution of mathematical formulas and algorithms regardless of their complexity or the need for 
specialized knowledge.  The system can translate natural language problem statements into symbolic representations,
allowing the reasoning module to perform calculations, and then translate the result back into natural language.

2. **Knowledge Integration for Contextual Understanding:**  LLMs may lack access to specific data or formulas 
necessary for advanced mathematical tasks.  MRKL's integration of external knowledge sources (databases, 
specialized libraries, etc.) provides access to this information, enabling the system to correctly interpret the 
context of a problem and select the appropriate formulas and data.

3. **Modular Design for Flexibility and Scalability:** The modular design of MRKL allows for the addition of new 
knowledge modules and reasoning capabilities as needed. This flexibility makes it adaptable to various mathematical
domains and facilitates easier expansion for tackling more intricate problems.

4. **Hybrid Approach for Enhanced Performance:**  MRKL's combination of LLMs and symbolic reasoning creates a 
synergistic effect. LLMs can be used for initial problem understanding, formula extraction from textual 
descriptions, and final result interpretation, while the symbolic modules handle the computationally intensive 
parts.

However, challenges remain. The text mentions difficulties in intelligently routing inputs between modules and 
avoiding model explosion.  Future development of MRKL should focus on optimizing these aspects to improve 
efficiency and scalability for even more complex mathematical problems.  Furthermore, careful consideration needs 
to be given to the selection and integration of external knowledge sources to ensure accuracy and reliability of 
the information used.

Q 3

Question:  Given the ReAct model's demonstrated success in mitigating hallucination and error propagation in 
question answering and fact verification by interacting with a Wikipedia API, what novel applications could be 
explored by integrating ReAct with other external knowledge bases or APIs, and how might this impact the model's 
reasoning and action capabilities across diverse tasks?

RAG Answer: 

Answer: Integrating ReAct with various external knowledge bases and APIs beyond Wikipedia could unlock numerous 
novel applications and significantly enhance its reasoning and action capabilities across diverse tasks.  The key 
is ReAct's ability to synergistically combine reasoning and action, allowing it to iteratively refine its 
understanding and actions based on external feedback.

Here are some examples:

**1.  Scientific Literature Retrieval and Reasoning:**  Integrating ReAct with APIs for scientific databases like 
PubMed, Semantic Scholar, or arXiv could enable the model to answer complex scientific questions requiring 
multi-step reasoning across multiple papers.  This would allow for more accurate and nuanced answers than current 
LLMs, which often hallucinate facts or misinterpret scientific findings.  The impact on reasoning would be a more 
robust and evidence-based approach, and the action capability would extend to retrieving and synthesizing 
information from disparate sources.

**2.  Financial Market Analysis and Prediction:**  Connecting ReAct to financial data APIs (e.g., Alpha Vantage, 
Tiingo) would enable sophisticated financial analysis.  The model could reason through complex market trends, 
analyze company performance, and even formulate potential investment strategies by accessing and interpreting 
real-time market data. Reasoning would become more data-driven and precise, while actions would involve querying 
APIs for specific data points and formulating trades (in a simulated environment initially).

**3.  Legal Research and Argumentation:**  Integration with legal databases like Westlaw or LexisNexis would 
revolutionize legal research.  ReAct could analyze case law, statutes, and regulations to provide accurate and 
comprehensive legal opinions.  The reasoning capabilities would improve by utilizing precise legal language and 
precedents, while actions would include retrieving relevant legal documents and constructing logical legal 
arguments.

**4.  Medical Diagnosis and Treatment Planning (Simulated Environment):**  Within a simulated environment (crucial 
for safety), ReAct could be integrated with medical knowledge bases and imaging APIs.  This could assist medical 
professionals by analyzing patient data, suggesting potential diagnoses, and even outlining possible treatment 
plans.  Reasoning would involve complex medical knowledge integration and risk assessment, while actions could 
involve querying medical databases and creating treatment plans.  Ethical considerations and validation would be 
paramount in any real-world application.


**5.  Personalized Education and Tutoring:**  By connecting ReAct to educational APIs and databases, personalized 
learning experiences could be created.  The model could adapt its teaching style to individual student needs, 
provide targeted feedback, and generate custom learning materials.  Reasoning would involve understanding the 
student's knowledge gaps and tailoring lessons accordingly, while actions would include selecting relevant learning
resources and providing personalized feedback.


In all these examples, the impact on ReAct's capabilities would be significant.  The iterative process of reasoning
and acting, guided by external information, would mitigate hallucination, improve accuracy, and produce more robust
and explainable outputs. The ability to handle complex, multi-step reasoning tasks across diverse domains would be 
significantly enhanced.  However, careful consideration of ethical implications, bias in data sources, and the 
potential for misuse remains crucial for any real-world deployment.

### 5.5. Implementing Human Preference Metric

Now we implement the fourth part of our evaluation routine:

- Sample RAG agent document pool to find two document chunks.
- Use those two document chunks to generate synthetic "baseline" question-answer pair.
- Use RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests if the RAG chain outperforms a narrow chatbot with limited document access.

In [252]:
from ragas import evaluate, EvaluationDataset, SingleTurnSample
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LangchainLLMWrapper

eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for accuracy & precision of detail.
[1] First answer is more detailed, accurate and precise.
[2] Second answer is more detailed, accurate and precise.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []
trio_gen = zip(synth_questions, synth_answers, rag_answers)

# For RAGAs scores
samples = [
    SingleTurnSample(user_input=q, retrieved_contexts=[c], response=a_rag, reference=a_synth)
    for q, c, a_rag, a_synth in zip(synth_questions, contexts, rag_answers, synth_answers)
]

dataset = EvaluationDataset(samples)

# evaluator_llm = LangchainLLMWrapper(instruct_llm)
# evaluator_embeddings = LangchainEmbeddingsWrapper(embedder)

# metrics = [LLMContextRecall(llm=evaluator_llm), FactualCorrectness(llm=evaluator_llm), Faithfulness(llm=evaluator_llm), SemanticSimilarity(embeddings=evaluator_embeddings)]
# results = evaluate(dataset=dataset, metrics=metrics)

for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")
    qa_trio = f"Question: {q}\n\nAnswer 1: {a_rag}\n\n Answer 2: {a_synth}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")
# pprint2(f"RAGAs Scores: {result.scores.samples.todict()}\n\n")

Set 1

Question: Question:  Considering the ReAct model's success in integrating reasoning and action, as exemplified by 
its ability to plan and execute actions like cleaning and placing a lettuce in the provided example, how might its 
interactive decision-making capabilities be further enhanced by incorporating more sophisticated environmental 
modeling and proactive planning beyond simple task completion?

RAG Answer: Answer: Enhancing ReAct's interactive decision-making capabilities beyond simple task completion 
requires incorporating more sophisticated environmental modeling and proactive planning.  This could involve:

1. **Developing richer internal representations of the environment:**  Instead of relying on simple descriptions, 
ReAct could benefit from integrating more detailed, structured environmental models. This might involve using 
graph-based representations to capture object relationships, spatial layouts, and potential interactions.  This 
would allow for more complex reasoning about the consequences of actions and more robust planning.

2. **Implementing proactive planning mechanisms:** Currently, ReAct's planning seems largely reactive.  More 
sophisticated planning algorithms, such as Monte Carlo Tree Search (MCTS) or hierarchical planning, could be 
incorporated to explore potential action sequences and evaluate their expected outcomes before execution.  This 
proactive approach would enable anticipation of future needs and potential obstacles, leading to more efficient and
robust task completion.

3. **Integrating temporal reasoning:**  Adding temporal reasoning capabilities would allow ReAct to consider the 
timing of actions and their dependencies. For instance, understanding that cleaning the lettuce must precede 
placing it on the table is a simple example; more complex scenarios may require sophisticated scheduling and 
resource management.

4. **Incorporating uncertainty and risk assessment:**  Real-world environments are inherently uncertain.  
Augmenting ReAct with probabilistic models and risk assessment mechanisms would allow it to handle unexpected 
events and make more robust decisions under uncertainty. For example, it could plan for alternative actions if a 
required object is unavailable.

5. **Leveraging external knowledge bases:**  Integrating access to broader knowledge bases beyond simple APIs could
provide ReAct with more context and information for planning.  This could include knowledge about object 
properties, typical action sequences, or potential hazards.


By integrating these enhancements, ReAct could move beyond simple task completion towards more complex, adaptive, 
and proactive interaction with the environment.  It would then be able to handle unforeseen circumstances, plan for
contingencies, and optimize its actions for efficiency and robustness.

Synth Answer: Answer: Enhancing ReAct's interactive decision-making capabilities beyond simple task completion 
requires incorporating more sophisticated environmental modeling and proactive planning. This could involve:  1) 
**Developing richer world models:**  Instead of relying on limited, immediate sensory input, integrating external 
knowledge bases, maps, or object-relationship graphs would allow ReAct to reason about the environment more 
comprehensively. This would enable proactive actions like anticipating obstacles before they are encountered (e.g.,
planning a route around a cluttered area before starting to clean). 2) **Hierarchical planning:**  Moving beyond 
flat sequences of actions, hierarchical planning would allow ReAct to break down complex tasks into sub-tasks, 
enabling more robust and flexible planning.  This would allow for contingency planning –  if an unexpected event 
occurs (e.g., the lettuce is unexpectedly slippery), the system can adapt its plan without restarting. 3) 
**Predictive modeling:**  Incorporating predictive models of the environment and the effects of actions would allow
ReAct to simulate potential outcomes and choose actions that maximize the likelihood of success. For example, 
predicting the trajectory of a moving object to avoid collisions while navigating. 4) **Reinforcement learning with
long-term rewards:**  Training the model with rewards that incentivize efficient and proactive behavior, extending 
beyond immediate task completion, would encourage the system to develop more sophisticated planning skills.  For 
example, rewarding efficient route planning or minimizing energy consumption. 5) **Uncertainty handling:**  
Incorporating probabilistic reasoning and uncertainty quantification would make the system more robust to noisy or 
incomplete information about the environment. This would allow ReAct to handle situations where perfect information
is unavailable and plan accordingly.  By incorporating these advancements, ReAct could evolve from a simple task 
executor to a proactive and adaptable agent capable of complex, long-term planning and decision-making in dynamic 
environments.

Synth Evaluation: [1] Justification:

Answer 1 provides a more precise and detailed breakdown of how to enhance ReAct's capabilities.  While Answer 2 
touches upon similar concepts, Answer 1 offers more specific examples and suggestions. For instance, Answer 1 
explicitly mentions Monte Carlo Tree Search (MCTS) and hierarchical planning as specific algorithmic solutions, 
offering a greater level of technical depth.  The inclusion of temporal reasoning and the explicit mention of 
handling uncertainty with probabilistic models adds further precision to Answer 1's recommendations. Answer 2's 
points are more broadly stated, lacking the same level of specific detail in its suggestions for improvement.  
Therefore, Answer 1 is superior in terms of accuracy and precision.

Set 2

Question: Question: Given the limitations of Large Language Models highlighted in the provided text, and the 
presented data on the performance of different mathematical formula structures within the MRKL system, how might 
the specific strengths and weaknesses of neuro-symbolic architectures like MRKL be leveraged to improve the 
accuracy and reliability of complex mathematical reasoning tasks in AI systems?

...

Synth Evaluation: [1] Justification: Answer 1 provides a more precise and detailed explanation of how MRKL's 
specific features address the limitations of LLMs in mathematical reasoning.  It offers concrete examples (e.g., 
translation between natural language and symbolic representations) and directly connects the MRKL system's 
components (LLMs, external knowledge sources, reasoning modules) to the solution of improving accuracy and 
reliability.  Answer 2, while offering valid points, remains more general and less specific in its connection to 
MRKL's architecture and functionality.  For example, it mentions "improved knowledge representation" without 
explicitly detailing how MRKL's structure facilitates this, unlike Answer 1's explicit discussion of symbolic 
reasoning modules and knowledge integration.  Answer 1 also more thoroughly addresses potential challenges and 
future development directions, thus showing a more complete understanding of the subject matter.

Set 3

Question: Question:  Given the ReAct model's demonstrated success in mitigating hallucination and error propagation
in question answering and fact verification by interacting with a Wikipedia API, what novel applications could be 
explored by integrating ReAct with other external knowledge bases or APIs, and how might this impact the model's 
reasoning and action capabilities across diverse tasks?

RAG Answer: Answer: Integrating ReAct with various external knowledge bases and APIs beyond Wikipedia could unlock 
numerous novel applications and significantly enhance its reasoning and action capabilities across diverse tasks.  
The key is ReAct's ability to synergistically combine reasoning and action, allowing it to iteratively refine its 
understanding and actions based on external feedback.

Here are some examples:

**1.  Scientific Literature Retrieval and Reasoning:**  Integrating ReAct with APIs for scientific databases like 
PubMed, Semantic Scholar, or arXiv could enable the model to answer complex scientific questions requiring 
multi-step reasoning across multiple papers.  This would allow for more accurate and nuanced answers than current 
LLMs, which often hallucinate facts or misinterpret scientific findings.  The impact on reasoning would be a more 
robust and evidence-based approach, and the action capability would extend to retrieving and synthesizing 
information from disparate sources.

**2.  Financial Market Analysis and Prediction:**  Connecting ReAct to financial data APIs (e.g., Alpha Vantage, 
Tiingo) would enable sophisticated financial analysis.  The model could reason through complex market trends, 
analyze company performance, and even formulate potential investment strategies by accessing and interpreting 
real-time market data. Reasoning would become more data-driven and precise, while actions would involve querying 
APIs for specific data points and formulating trades (in a simulated environment initially).

**3.  Legal Research and Argumentation:**  Integration with legal databases like Westlaw or LexisNexis would 
revolutionize legal research.  ReAct could analyze case law, statutes, and regulations to provide accurate and 
comprehensive legal opinions.  The reasoning capabilities would improve by utilizing precise legal language and 
precedents, while actions would include retrieving relevant legal documents and constructing logical legal 
arguments.

**4.  Medical Diagnosis and Treatment Planning (Simulated Environment):**  Within a simulated environment (crucial 
for safety), ReAct could be integrated with medical knowledge bases and imaging APIs.  This could assist medical 
professionals by analyzing patient data, suggesting potential diagnoses, and even outlining possible treatment 
plans.  Reasoning would involve complex medical knowledge integration and risk assessment, while actions could 
involve querying medical databases and creating treatment plans.  Ethical considerations and validation would be 
paramount in any real-world application.


**5.  Personalized Education and Tutoring:**  By connecting ReAct to educational APIs and databases, personalized 
learning experiences could be created.  The model could adapt its teaching style to individual student needs, 
provide targeted feedback, and generate custom learning materials.  Reasoning would involve understanding the 
student's knowledge gaps and tailoring lessons accordingly, while actions would include selecting relevant learning
resources and providing personalized feedback.


In all these examples, the impact on ReAct's capabilities would be significant.  The iterative process of reasoning
and acting, guided by external information, would mitigate hallucination, improve accuracy, and produce more robust
and explainable outputs. The ability to handle complex, multi-step reasoning tasks across diverse domains would be 
significantly enhanced.  However, careful consideration of ethical implications, bias in data sources, and the 
potential for misuse remains crucial for any real-world deployment.

Synth Answer: Answer: Integrating ReAct with other external knowledge bases and APIs beyond Wikipedia could unlock 
numerous novel applications and significantly enhance its reasoning and action capabilities.  Here are some 
examples:

**Novel Applications:**

* **Scientific Discovery and Hypothesis Generation:**  ReAct could be integrated with scientific databases like 
PubMed, Reaxys, or arXiv.  This would allow it to formulate hypotheses, design experiments, and interpret results 
by referencing relevant literature and data, potentially accelerating scientific breakthroughs.  The model could 
reason about complex biological pathways, chemical reactions, or astronomical phenomena by accessing and processing
information from these specialized sources.

* **Financial Modeling and Risk Assessment:** Combining ReAct with financial APIs (e.g., accessing real-time market
data, company financials, economic indicators) could enable sophisticated financial modeling, risk assessment, and 
investment strategy generation. The model could analyze market trends, predict future performance, and make 
informed investment decisions based on the data it retrieves and processes.

* **Personalized Healthcare and Medical Diagnosis Support:**  Integrating ReAct with Electronic Health Records 
(EHR) systems and medical knowledge bases could assist doctors in diagnosis and treatment planning.  The model 
could analyze patient data, identify potential diagnoses, suggest relevant treatments, and even anticipate 
potential complications based on its access to medical literature and patient histories. However, ethical 
considerations regarding patient privacy and data security would need careful attention.

* **Legal Research and Due Diligence:**  Access to legal databases (e.g., Westlaw, LexisNexis) would allow ReAct to
perform legal research, identify relevant case law, and assist in due diligence processes. This could significantly
streamline legal workflows and improve the accuracy of legal advice.

* **Supply Chain Optimization and Logistics:**  Integration with real-time logistics and supply chain data APIs 
could allow ReAct to optimize inventory management, predict disruptions, and suggest efficient routing strategies. 
This could lead to cost savings and increased efficiency in supply chains.

**Impact on Reasoning and Action Capabilities:**

Integrating ReAct with diverse external knowledge bases would:

* **Expand its knowledge domain:**  ReAct's capabilities would no longer be limited to the information available 
within Wikipedia.  It could access and process information from specialized domains, significantly broadening its 
scope.
* **Improve reasoning accuracy:**  Access to more comprehensive and up-to-date information would reduce 
hallucination and improve the accuracy of its reasoning and conclusions.
* **Enable more complex reasoning tasks:**  The ability to interact with multiple data sources would allow ReAct to
perform more complex reasoning tasks that require synthesizing information from diverse sources.
* **Enhance action capabilities:** The model's ability to directly interact with and manipulate external systems 
(e.g., making API calls to update databases, trigger actions in a robotic system) would enable more sophisticated 
actions.


However, challenges remain: ensuring data quality, managing API rate limits, dealing with conflicting information 
from different sources, and addressing potential biases in the data accessed are crucial considerations for 
successful integration.

Synth Evaluation: [1] Justification: Answer 1 provides more specific and detailed examples of novel applications of
ReAct integrated with various APIs.  Each example (scientific literature, finance, legal research, simulated 
medical diagnosis, personalized education) is clearly explained, including how the integration would impact both 
the reasoning and action capabilities of the model.  Answer 2 offers good examples as well, but they are less 
detailed in explaining the impact on reasoning and action.  For example, "Scientific Discovery and Hypothesis 
Generation" is mentioned but lacks the granular explanation provided by Answer 1's "Scientific Literature Retrieval
and Reasoning."  Answer 1's inclusion of ethical considerations in each example further enhances its precision and 
completeness. While Answer 2 mentions challenges, Answer 1's discussion of ethical implications adds a crucial 
layer of detail regarding responsible implementation.

We now have an LLM system that reasons about our pipeline and tries to evaluate it. Now we have some judge results, we aggregate the results and see how often our formulation was correct according to an LLM:

In [254]:
pref_score = sum(("[1]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 1.0
